# Dynamic Topic Modeling with YouTube Comments

We will be using Dynamic Topic Modeling with BERTopic to visualize how topics in YouTube videos have evolved over time. These topics will be visualized and thoroughly explored. 

## Dynamic Topic Models
Dynamic topic models can be used to analyze the evolution of topics of a collection of documents over time. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Installing BERTopic

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# **Data**
We have taken comments from YuTube videos related to AI/ML/Dl/DS. This includes comments and reply.

Moreover, since we are looking at comments over time, we will be saving all timestamps related to comments.

In [ ]:
import re
import pandas as pd
from datetime import datetime

# Load data
df = pd.read_csv('/content/drive/MyDrive/YouTube-Project/data-youtube-comments.csv')

# Filter
df.comment = df.apply(lambda row: re.sub(r"http\S+", "", row.comment).lower(), 1)
df.text = df.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.comment.split())), 1)
df.text = df.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.comment).split()), 1)
timestamps = df.date.to_list()
comments = df.comment.to_list()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
comments[25]

'thank you for this great tutorial. can you please share the data sets on nastia.tetervak@<a href=" thank you! :)'

# **Dynamic Topic Modeling**


## Basic Topic Model
To perform Dynamic Topic Modeling with BERTopic we will first need to create a basic topic model using all comments. The temporal aspect will be ignored as we are, for now, only interested in the topics that reside in those comments. 

In [ ]:
pip install numpy==1.19.2

     |████████████████████████████████| 14.5 MB 32 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.1
    Uninstalling numpy-1.21.1:
      Successfully uninstalled numpy-1.21.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
bertopic 0.9.0 requires numpy>=1.20.0, but you have numpy 1.19.2 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
from bertopic import BERTopic
topic_model = BERTopic(min_topic_size=35, verbose=True)
topics, _ = topic_model.fit_transform(comments)

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


2021-08-10 19:07:33,860 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2021-08-10 19:09:55,153 - BERTopic - Reduced dimensionality with UMAP
2021-08-10 19:10:05,125 - BERTopic - Clustered UMAP embeddings with HDBSCAN


We can then extract most frequent topics:

In [ ]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name
0,-1,21475,-1_bra_cost_br_layer
1,0,2291,0_button_subscribing_link_sent
2,1,1729,1_python_programming_started_learn
3,2,1612,2_explanation_explained_explaination_explaining
4,3,1367,3_scientist_degree_career_job
5,4,996,4_pca_pc1_pc2_pcs
6,5,763,5_math_calculus_maths_algebra
7,6,762,6_datasets_sets_set_send
8,7,755,7_send_datasets_plz_pls
9,8,636,8_tutorials_tutor_tutorial_beginner


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [ ]:
topic_nr = freq.iloc[3]["Topic"]  # We select a frequent topic
topic_model.get_topic(topic_nr)   # You can select a topic number as shown above

[('explanation', 0.02745347611828932),
 ('explained', 0.018780780788532),
 ('explaination', 0.009528449261588329),
 ('explaining', 0.009107228067583783),
 ('explanations', 0.00858001643675596),
 ('concepts', 0.006580730387566006),
 ('explain', 0.005738608166675469),
 ('concept', 0.00564859871078422),
 ('complex', 0.005172753318640418),
 ('complicated', 0.004563966918520288)]

We can visualize the basic topics that were created with the Intertopic Distance Map. This allows us to judge visually whether the basic topics are sufficient before proceeding to creating the topics over time. 

In [ ]:
fig = topic_model.visualize_topics(); fig

## Topics over Time
Before we start with the Dynamic Topic Modeling step, it is important that you are satisfied with the topics that were created previously. We are going to be using those specific topics as a base for Dynamic Topic Modeling. 

Thus, this step will essentially show you how the topics that were defined previously have evolved over time. 

There are a few important parameters that you should take note of, namely:

* `docs`
  * These are the comments that we are using
* `topics`
  * The topics that we have created before
* `timestamps`
  * The timestamp of each comment/document
* `global_tuning`
  * Whether to average the topic representation of a topic at time *t* with its global topic representation
* `evolution_tuning`
  * Whether to average the topic representation of a topic at time *t* with the topic representation of that topic at time *t-1*
* `nr_bins`
  * The number of bins to put our timestamps into. It is computationally inefficient to extract the topics at thousands of different timestamps. Therefore, it is advised to keep this value below 20. 


In [ ]:
topics_over_time = topic_model.topics_over_time(docs=comments, 
                                                topics=topics, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

20it [19:29, 58.47s/it]


## Visualize Topics over Time
After having created our `topics_over_time`, we will have to visualize those topics as accessing them becomes a bit more difficult with the added temporal dimension. 

To do so, we are going to visualize the distribution of topics over time based on their frequency. Doing so allows us to see how the topics have evolved over time. Make sure to hover over any point to see how the topic representation at time *t* differs from the global topic representation. 


In [ ]:
topic_model.visualize_topics_over_time(topics_over_time)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, topics=[1, 4, 5, 13, 14, 17, 19])